트리 알고리즘

1. [결정 트리](#결정-트리)
    - [로지스틱 회귀로 와인 분류하기](#로지스틱-회귀로-와인-분류하기)
    - [결정 트리](#결정-트리)

# 결정 트리

## 로지스틱 회귀로 와인 분류하기

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

와인 데이터를 먼저 불러오자. 

앞에서 열 3개는 각각 알코올 도수,당도,PH 값을 나타낸다. 네번째 열은 타깃값으로 0이면 레드와인 1이면 화이트 와인이다. 

In [ ]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine.head()

레드 와인과 화이트 와인을 구분하는 이진 분류 문제로 즉 전체 와인에서 화이트 와인을 골라내는 문제이다. 

In [ ]:
wine.info()

In [ ]:
wine.describe()

알코올 도수와 당도, PH 값의 스케일이 다르다는 것이다.


In [ ]:
wine.hist(bins=20)
plt.show()


데이터프레임을 넘파이 배열로 바꾸고 훈련 세트와 테스트 세트로 나누자.

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

### 학습/검증 세트

샘플개수가 충분히 많으므로 20% 정도만 테스트 세트로 나누었다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)

알코올 도수와 당도, PH 값의 스케일이 다른 범위 값을 StandardScaler 클래스를 사용해 특성을 표준화하자. 

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(X_train)

X_scaled = ss.transform(X_train)
XXtest_scaled = ss.transform(X_test)

이제 표준점수로 변환된 X_scaled와 Xtest_scaled를 사용해 로지스틱 회귀 모델을 훈련하자.


In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_scaled, y_train)

print(lr.score(X_scaled, y_train))
print(lr.score(XXtest_scaled, y_test))

점수가 높지 않다. ***훈련 세트와 테스트 세트의 점수가 모두 낮아서 모델이 과소적합*** 이라 볼 수 있다.이 문제를 해결하기 위해 규제 매개 변수 C의 값을 바꾸던지 solver 매개변수에서 다른 알고리즘을 선택할 수도 있다. 또한 다항 특성을 만들어 추가할 수도 있다.

로지스틱 회귀가 학습한 계수와 절편을 출력해 보자.

In [ ]:
print(lr.coef_, lr.intercept_)

사실 이 모델이 왜 저런 계수 값을 학습했는지 정확히 이해하기 어렵다. 

아마도 알코올 도수와 당도가 높을수록 화이트 와인일 가능성이 높고, PH가 높을수록 레드 와인일 가능성이 높은 것 같다. 하지만 정확히 이 숫자가 어떤 의미인지 설명하긴 어렵다. 

## 결정 트리

결정 트리 모델은 스무고개와 같다. 데이터를 잘 나눌 수 있는 질문을 찾는다면 계속 질문을 추가해서 분류 정확도를 높일 수 있다. 


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_scaled, y_train)

print(dt.score(X_scaled, y_train))
print(dt.score(XXtest_scaled, y_test))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(10,7))
plot_tree(dt)
plt.show()

맨 위의 노드를 루프노드라 부르고 맨 아래 끝에 달린 노드를 리프 노드라고 한다.

너무 복잡하니 plot_tree() 함수에서 트리의 깊이를 제한해서 출력해 보자.

In [ ]:
plt.figure(figsize=(10,7))
plot_tree(dt, max_depth=1, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

루트 노드는 sugar가 -0.239 이하인지 질문을 한다. 왼쪽이 yes 오른쪽이 no이다.  만약 어떤 샘플의 당도가 -0.239와 같거나 작으면 왼쪽 가지로 이동한다. 그렇지 않으면 오른쪽으로 이동한다

이번엔 노드 상자 안에 있는 gini에 대해 알아보자.

### - 불순도
gini는 지니 불순도(gini impurity)를 의미한다. DecisionTreeClassifier 클래스의 criterion 매개변수의 기본값이 gini이다. creterion 매개변수의 용도는 노드에서 데이터를 분할할 기준을 정하는 것이다. 지니 불순도는 클래스의 비율을 제곱해서 더한 다음 1에서 빼면된다.

```
지니 불순도 = 1 - (음성 클래스 비율의 제곱 + 양성 클래스 비율의 제곱)
```

### - 가지치기

DecisionTreeClassifier 클래스의 max_depth 매개변수를 3으로 지정하여 모델을 만들어 보자.

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(X_scaled, y_train)

print(dt.score(X_scaled, y_train))
print(dt.score(Xtest_scaled, y_test))

훈련 세트의 성능은 낮아졌지만 테스트 세트의 성능은 거의 그대로이다. plot_tree() 함수로 그려보자.

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

앞서 불순도를 기준으로 샘플을 나눌때 특성값의 스케일은 결정 트리 알고리즘에 아무런 영향을 미치지 않는다. 따라서 표준화 전처리를 할 필요가 없다. 이것이 결정 트리 알고리즘의 또 다른 장점 중 하나이다.

그럼 앞서 전처리하기 전의 훈련 세트와 테스트 세트로 결정 트리모델을 다시 훈련해 보자.

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(X_train, y_train)

print(dt.score(X_train, y_train))
print(dt.score(X_test, y_test))

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

특성값을 표준점수로 바꾸지 않은 터라 이해하기가 훨씬 쉽다. 당도가 1.625보다 크고 4.325보다 작은 와인중 알코올 도수가 11.025와 같거나 작은 것이 레드 와인이다. 그 이외에는 모두 화이트 와인으로 예측했다.


#### 특성 중요도

마지막으로 결정 트리는 어떤 특성이 가장 유용한지 나타내는 특성 중요도를 계산해 준다. 특성 중요도는 결정 트리 모델의 featureimportances 속성에 저장되어 있다. 이 값을 출력해 보자.

In [ ]:
print(dt.feature_importances_)

역시 두 번째 특성인 sugar가 0.87 정도로 특성 중요도가 가장 높다. 특성 중요도는 각 노드의 정보 이득과 전체 샘플에 대한 비율을 곱한 후 특성별로 더하여 계산한다. 특성 중요도를 활용하면 결정 트리 모델을 특성 선택에 활용할 수 있다. 이것이 결정 트리 알고리즘의 또 다른 장점중 하나이다.

## 확인문제

불순물의 기준을 변경해 보자.

In [ ]:
dt = DecisionTreeClassifier(min_impurity_decrease=0.0005, random_state=42)
dt.fit(X_train, y_train)

print(dt.score(X_train, y_train))
print(dt.score(X_test, y_test))

In [ ]:
plt.figure(figsize=(20,15), dpi=300)
plot_tree(dt, max_depth=3, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

---

참고
 - 혼자 공부하는 머신러닝+딥러닝